# BUSINESS UNDERSTANDING

## Summary

- 주제: ***비식별화 된 시스템 기록(로그 및 수치 데이터)를 분석***하여 시스템 품질 변화로 사용자에게 불편을 야기하는 요인을 진단

- 배경: 다양한 장비/서비스에서 일어나는 시스템 데이터를 통해 사용자의 불편을 예지하기 위해 **'시스템 데이터'** 와 **'사용자 불편 발생 데이터'** 를 분석하여 ***불편을 느낀 사용자*** 와 ***불편 요인들*** 을 찾아주세요.

- 목적
    1. 데이터를 통해 ***사용자가 불편을 느끼는 원인 분석***
    2. ***사용자 관점*** 의 데이터 분석 능력이 뛰어난 인재 발굴

## Rule

1차 평가

1. Evaluation Metric: AUC(사용자로부터 불만 접수가 일어날 확률 예측)
2. Public score: 전체 테스트 데이터 중 33%
3. Private score: 전체 테스트 데이터 중 67%
4. 1차 평가의 최종 순위는 Private score

2차 평가

   1. 결과분석, 비즈니스 분석
   2. 사용자 불만 접수 원인 분석
   3. err_data의 err간 관계 해석
   4. quality_data 수치 해석
   5. err_data와 quality_data간의 관계 해석 필수 포함

기타

1. 외부데이터 금지
2. 사전학습모델 금지
3. 제공 된 학습 데이터 외 학습 불가
4. 테스트 데이터는 추론 과정에서만 사용 가능(train 불가)

## Data Describtion

학습 데이터 (user_id: 10,000 ~ 24,999, 15,000명)

- train_e: 시스템에 발생한 **에러 로그**
- train_q: 시스템 **퀄리티 로그**
- train_p: 사용자 불만 및 불만이 접수된 시간

테스트 데이터(user_id: 30,000 ~ 44,998, 14999명)

- test_e: 상동
- test_q: 상동
- 테스트 데이터에는 test_p 테이블이 없음

## Discussion

주어진 기간 내에 err가 발생하지 않은 경우, err_set에 해당 user_id가 존재하지 않음. 즉 train_err, test_err 에 user_id가 없으면 err이 발생하지 않은 것임

# IMPORT & LOAD DATA

In [147]:
import load_dtypes as ld
import warnings
warnings.filterwarnings(action='ignore')

import os, sys

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
TRAIN_P_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv'
TRAIN_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv'
TRAIN_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv'
TEST_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv'
TEST_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/sample_submission.csv'

In [221]:
%%time
train_p = ld.load_dtypes(TRAIN_P_PATH)
train_q = ld.load_dtypes(TRAIN_Q_PATH)
train_e = ld.load_dtypes(TRAIN_E_PATH)
test_q = ld.load_dtypes(TEST_Q_PATH)
test_e = ld.load_dtypes(TEST_E_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv
Wall time: 1min 16s


# DATA UNDERSTANDING

각 데이터 테이블의 이름이 너무 길기 때문에 아래와 같이 정의하겠습니다.

- train_problem_data -> train_p
- train_quality_data -> train_q
- train_err_data -> train_e
- test_quality_data -> test_q
- test_err_data -> test_e

## Data Table Opimization

### train_p optimization

In [222]:
train_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int16
 1   time     5429 non-null   int64
dtypes: int16(1), int64(1)
memory usage: 53.1 KB


user당 problem이 한개가 아닙니다.<br>
user_id 를 count 해서 target으로 넣어주겠습니다. null user를 0으로 넣어주어야겠습니다.<br>
그리고 time을 x_train으로 붙여서 정보를 얻어봐야겠습니다.

In [223]:
# 10001부터 24999까지의 index를 만들어줍니다.
user_id_idx = np.array(range(10001, 25000, 1))

# train_new_p라는 새로운 df를 만들고 index는 위에서 만든 user_id_idx 로 지정해줍니다.
train_new_p = pd.DataFrame(index = user_id_idx)

# target이라는 칼럼을 만들고, train_p를 user_id를 groupby해서 time을 count한 값을 넣어줍니다.
# index가 지정되어 있기 때문에, 해당 값에만 들어갑니다.
train_new_p['target'] = train_p.groupby('user_id')['time'].count()

# train_p에 존재하지 않는 user_id는 값을 받지 못해 Nan값이므로, 에러가 나지 않았다는 뜻의 0으로 채워줍니다.
train_new_p = train_new_p.fillna(0)

In [225]:
train_new_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14999 entries, 10001 to 24999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  14999 non-null  float64
dtypes: float64(1)
memory usage: 234.4 KB


In [227]:
# 기술통계를 살펴보면, max가 5로 되어있습니다.
# 일부러 {0,1} 의 이진분류로 값을 넣지 않고, 0~5의 int 값으로 넣었습니다.
# 회귀 문제로 가져갈 수도 있습니다.
train_new_p.describe().T

,count,mean,std,min,25%,50%,75%,max
target,14999.0,0.361957,0.544458,0.0,0.0,0.0,1.0,5.0


In [228]:
train_new_p.head()

,target
10001,1.0
10002,0.0
10003,0.0
10004,1.0
10005,1.0


### train_q optimization

In [170]:
train_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   time        828624 non-null  int64   
 1   user_id     828624 non-null  int16   
 2   fwver       788544 non-null  category
 3   quality_0   684192 non-null  float32 
 4   quality_1   828624 non-null  int16   
 5   quality_2   788511 non-null  float32 
 6   quality_3   828624 non-null  int8    
 7   quality_4   828624 non-null  int8    
 8   quality_5   828604 non-null  category
 9   quality_6   828624 non-null  int16   
 10  quality_7   828624 non-null  category
 11  quality_8   828624 non-null  category
 12  quality_9   828624 non-null  category
 13  quality_10  828624 non-null  category
 14  quality_11  828624 non-null  int8    
 15  quality_12  828624 non-null  int8    
dtypes: category(6), float32(2), int16(3), int64(1), int8(4)
memory usage: 28.9 MB


살펴본 결과 q 데이터는 전부 category라고 보는 게 맞는 것 같습니다. 형변환을 해주어야겠습니다.<br>
time도 데이트타임으로 짤라줍시다. 시간단위로 짜릅니다.

In [220]:
# 먼저 퀄리티 피쳐들을 리스트로 받아줍니다.
quality_features = ['quality_0', 'quality_1', 'quality_2',
       'quality_3', 'quality_4', 'quality_5', 'quality_6', 'quality_7',
       'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']

# test셋에도 써야 하니 함수로 만들겠습니다.
def quality_as_category(df):
    # 함수 정의: df를 받아서 quality features를 category로 형변환해서 반환하는 함수
    
    for col in quality_features:
        try: # category가 아니면 에러가 나니, try-except로 합니다.
            if df[col].dtype == 'category':
                pass
        except: # error가 나면 category가 아니라는 뜻으로, 형변환을 해줍니다.
            df[col] = df[col].astype('category')
            
    return df

In [229]:
train_q = quality_as_category(train_q)

In [230]:
train_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   time        828624 non-null  int64   
 1   user_id     828624 non-null  int16   
 2   fwver       788544 non-null  category
 3   quality_0   684192 non-null  category
 4   quality_1   828624 non-null  category
 5   quality_2   788511 non-null  category
 6   quality_3   828624 non-null  category
 7   quality_4   828624 non-null  category
 8   quality_5   828604 non-null  category
 9   quality_6   828624 non-null  category
 10  quality_7   828624 non-null  category
 11  quality_8   828624 non-null  category
 12  quality_9   828624 non-null  category
 13  quality_10  828624 non-null  category
 14  quality_11  828624 non-null  category
 15  quality_12  828624 non-null  category
dtypes: category(14), int16(1), int64(1)
memory usage: 25.0 MB


### train_e opimization

In [231]:
train_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   user_id   int16   
 1   time      int64   
 2   model_nm  category
 3   fwver     category
 4   errtype   int8    
 5   errcode   category
dtypes: category(3), int16(1), int64(1), int8(1)
memory usage: 236.9 MB


errtype은 category 입니다. 형변환 해줍니다.<br>
time도 마찬가지로 데이트타임으로 짤라줍니다. 시간단위입니다.

In [232]:
# errtype 변수는 category로 형변환 해줍니다.
train_e['errtype'] = train_e.errtype.astype('category')
train_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   user_id   int16   
 1   time      int64   
 2   model_nm  category
 3   fwver     category
 4   errtype   category
 5   errcode   category
dtypes: category(4), int16(1), int64(1)
memory usage: 236.9 MB


### test_q optimization

In [184]:
test_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   time        747972 non-null  int64   
 1   user_id     747972 non-null  uint16  
 2   fwver       725208 non-null  category
 3   quality_0   641388 non-null  float32 
 4   quality_1   747961 non-null  category
 5   quality_2   726857 non-null  float32 
 6   quality_3   747972 non-null  int8    
 7   quality_4   747972 non-null  int8    
 8   quality_5   747928 non-null  category
 9   quality_6   747972 non-null  int16   
 10  quality_7   747972 non-null  category
 11  quality_8   747972 non-null  category
 12  quality_9   747972 non-null  category
 13  quality_10  747972 non-null  category
 14  quality_11  747972 non-null  int8    
 15  quality_12  747972 non-null  int8    
dtypes: category(7), float32(2), int16(1), int64(1), int8(4), uint16(1)
memory usage: 25.4 MB


마찬가지입니다.

In [233]:
test_q = quality_as_category(test_q)

In [234]:
test_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   time        747972 non-null  int64   
 1   user_id     747972 non-null  uint16  
 2   fwver       725208 non-null  category
 3   quality_0   641388 non-null  category
 4   quality_1   747961 non-null  category
 5   quality_2   726857 non-null  category
 6   quality_3   747972 non-null  category
 7   quality_4   747972 non-null  category
 8   quality_5   747928 non-null  category
 9   quality_6   747972 non-null  category
 10  quality_7   747972 non-null  category
 11  quality_8   747972 non-null  category
 12  quality_9   747972 non-null  category
 13  quality_10  747972 non-null  category
 14  quality_11  747972 non-null  category
 15  quality_12  747972 non-null  category
dtypes: category(14), int64(1), uint16(1)
memory usage: 22.6 MB


### test_e optimization

In [186]:
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   user_id   uint16  
 1   time      int64   
 2   model_nm  category
 3   fwver     category
 4   errtype   int8    
 5   errcode   category
dtypes: category(3), int64(1), int8(1), uint16(1)
memory usage: 236.6 MB


In [235]:
test_e['errtype'] = test_e.errtype.astype('category')
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   user_id   uint16  
 1   time      int64   
 2   model_nm  category
 3   fwver     category
 4   errtype   category
 5   errcode   category
dtypes: category(4), int64(1), uint16(1)
memory usage: 236.6 MB


In [187]:
submission

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


## Unique Values Check

In [236]:
# category feature가 많으니 unique value를 체크해야 합니다.
# 그와 함께 value_counts()도 체크해서 imbalance를 체크해야 합니다.

def show_unique(train):
    # 함수정의: df를 받아서 각 칼럼에 대해 category는 unique()와 value_counts()[:10] 를 반환하는 함수
    
    columns = train.columns
    for col in columns:
        try: # 마찬가지로 try except를 씁니다.
            if (train[col].dtype == 'category'):
                print("----------------------------------------")
                print("{}'s Unique values count: ".format(col), train[col].nunique())
                print("{}'s Unique values: ".format(col))
                print(train[col].unique())
                print(train[col].value_counts()[:10])
            else:
                pass
        except:
            pass

In [237]:
show_unique(train_q)

----------------------------------------
fwver's Unique values count:  27
fwver's Unique values: 
[05.15.2138, 04.22.1750, 04.16.3553, 04.33.1261, 04.22.1778, ..., 04.16.3439, 04.73.2571, 05.15.2114, 04.16.3345, 04.22.1442]
Length: 28
Categories (27, object): [05.15.2138, 04.22.1750, 04.16.3553, 04.33.1261, ..., 04.73.2571, 05.15.2114, 04.16.3345, 04.22.1442]
05.15.2138    163236
04.22.1750    142032
04.33.1261    131340
04.16.3553    111996
03.11.1167    104148
04.33.1185     66024
04.22.1778     56472
04.22.1684      3420
09.17.1431      3384
04.16.3571      2232
Name: fwver, dtype: int64
----------------------------------------
quality_0's Unique values count:  753
quality_0's Unique values: 
[0.0, 2.0, -1.0, NaN, 1.0, ..., 122.0, 228.0, 305.0, 188.0, 2351.0]
Length: 754
Categories (753, float64): [0.0, 2.0, -1.0, 1.0, ..., 228.0, 305.0, 188.0, 2351.0]
 0.0    542790
-1.0    130828
 1.0      2097
 2.0      1252
 3.0       518
 4.0       410
 5.0       385
 6.0       358
 7.0       3

In [238]:
show_unique(test_q)

----------------------------------------
fwver's Unique values count:  22
fwver's Unique values: 
[04.33.1261, 05.15.2138, 04.22.1750, 04.22.1778, 04.16.3553, ..., 03.11.1149, 05.15.2120, 04.33.1125, 04.16.3439, 05.15.2114]
Length: 23
Categories (22, object): [04.33.1261, 05.15.2138, 04.22.1750, 04.22.1778, ..., 05.15.2120, 04.33.1125, 04.16.3439, 05.15.2114]
05.15.2138    142704
04.22.1750    141912
04.33.1261    121584
04.16.3553    109056
03.11.1167     84240
04.22.1778     59736
04.33.1185     57828
04.16.3571      2172
04.22.1684      1464
03.11.1149      1188
Name: fwver, dtype: int64
----------------------------------------
quality_0's Unique values count:  540
quality_0's Unique values: 
[0.0, -1.0, NaN, 2.0, 8.0, ..., 10249.0, 10627.0, 10452.0, 2011.0, 239.0]
Length: 541
Categories (540, float64): [0.0, -1.0, 2.0, 8.0, ..., 10627.0, 10452.0, 2011.0, 239.0]
 0.0    505710
-1.0    127001
 1.0      2319
 2.0      1567
 3.0       739
 4.0       454
 5.0       290
 6.0       212
 8

In [239]:
show_unique(train_e)

----------------------------------------
model_nm's Unique values count:  9
model_nm's Unique values: 
[model_3, model_2, model_0, model_1, model_7, model_4, model_5, model_8, model_6]
Categories (9, object): [model_3, model_2, model_0, model_1, ..., model_4, model_5, model_8, model_6]
model_1    5384491
model_0    4176279
model_2    3473254
model_3    1907438
model_4    1507802
model_8      36064
model_7      35356
model_5      32233
model_6       1746
Name: model_nm, dtype: int64
----------------------------------------
fwver's Unique values count:  37
fwver's Unique values: 
[05.15.2138, 04.33.1185, 04.33.1261, 04.22.1750, 04.22.1778, ..., 04.22.1656, 04.16.2641, 05.15.2114, 04.16.3345, 05.15.2092]
Length: 37
Categories (37, object): [05.15.2138, 04.33.1185, 04.33.1261, 04.22.1750, ..., 04.16.2641, 05.15.2114, 04.16.3345, 05.15.2092]
04.16.3553    5237816
04.22.1750    2874213
04.33.1261    2504871
05.15.2138    1906479
03.11.1167    1505659
04.22.1778    1293946
04.33.1185     9637

In [240]:
show_unique(test_e)

----------------------------------------
model_nm's Unique values count:  9
model_nm's Unique values: 
[model_1, model_2, model_3, model_0, model_4, model_5, model_6, model_7, model_8]
Categories (9, object): [model_1, model_2, model_3, model_0, ..., model_5, model_6, model_7, model_8]
model_1    5473847
model_0    4350571
model_2    3601258
model_3    1801452
model_4    1215796
model_5      35462
model_7      28694
model_8      24994
model_6        574
Name: model_nm, dtype: int64
----------------------------------------
fwver's Unique values count:  40
fwver's Unique values: 
[04.16.3553, 04.16.3571, 04.33.1261, 05.15.2138, 04.22.1750, ..., 10.22.1770, 10.22.1780, 04.16.3569, 04.22.1772, 04.22.1170]
Length: 40
Categories (40, object): [04.16.3553, 04.16.3571, 04.33.1261, 05.15.2138, ..., 10.22.1780, 04.16.3569, 04.22.1772, 04.22.1170]
04.16.3553    5326124
04.22.1750    2969638
04.33.1261    2607195
05.15.2138    1800062
04.22.1778    1368279
03.11.1167    1209675
04.33.1185     9878

## Null Data Check & Missing Value Check

특이합니다. 많이 특이합니다. 일단 y_train 값은 user의 problem이 접수된 시간을 보여줍니다. 그렇다보니 user당 problem이 한 개가 아닙니다. 여러개가 있을 수 있습니다.

그러나 submission은 yes or no로 제출해야 합니다. 심지어 그것을 확률로 제출해야 합니다. 즉 모든 user에 대해서 problem이 1개라도 발생할 확률을 구해야 합니다.

이건 어찌보면 classification일 수 있습니다. 그러나 regression도 될 수 있습니다. 0 ~ n 까지의 target data가 있다고 생각하면 되니까요.

데이터도 매우 특이합니다. user-time 으로 테이블이 구성되어 있고, 퀄리티는 category인 것 같은데, 시간별로 계속 변합니다.

에러 데이터도 마찬가지입니다. 다양한 err code가 있습니다. model_nm, fwver 은 뭔지도 모르겠습니다.

Null data와 missing value 도 매우 많습니다.

# Vanila Modeling

# DATA PREPROCESSING

## EDA

## Feature Engineering

# MODELING

## Comparing Vanila Model

## Hyper Parameter Tuning

## K-fold Cross Validation

## Ensemble & Stacking